In [1]:
import numpy as np
import pyccl as ccl
import matplotlib.pyplot as plt
import scipy.integrate as integrate
import scipy.interpolate as interpol
from pyccl._core import UnlockInstance
from scipy import signal

In [2]:
import BCMprofiles_cleanUp as BCMprofiles
BCMprofiles.__all__

/Users/lambe/Documents/GitHub/Oxford-Intern-2024/Oxford-Internship-Practices/BCMprofiles_cleanUp.py:61: SyntaxWarning: invalid escape sequence '\ '
  """Creating a class for the stellar density profile
/Users/lambe/Documents/GitHub/Oxford-Intern-2024/Oxford-Internship-Practices/BCMprofiles_cleanUp.py:228: SyntaxWarning: invalid escape sequence '\ '
  """Creating a class for the bound gas density profile where:
/Users/lambe/Documents/GitHub/Oxford-Intern-2024/Oxford-Internship-Practices/BCMprofiles_cleanUp.py:435: SyntaxWarning: invalid escape sequence '\e'
  """Combined profile for the stellar & ejected & bound gas & cdm components (ie- The BCM Model), with the truncated Navarro-Frenk-White (NFW) profile used to calculate the density profiles of the cold dark matter (cdm) component.
/Users/lambe/Documents/GitHub/Oxford-Intern-2024/Oxford-Internship-Practices/BCMprofiles_cleanUp.py:504: SyntaxWarning: invalid escape sequence '\e'
  """Combined profile for the stellar & ejected & bound g

('StellarProfile',
 'EjectedGasProfile',
 'BoundGasProfile',
 'CDMProfile',
 'BCMProfile',
 'CombinedAllBCMProfile')

In [3]:
help(ccl.halos.profiles.nfw.HaloProfileNFW)

Help on class HaloProfileNFW in module pyccl.halos.profiles.nfw:

class HaloProfileNFW(pyccl.halos.profiles.profile_base.HaloProfileMatter)
 |  HaloProfileNFW(self, *, mass_def, concentration, fourier_analytic=True, projected_analytic=False, cumul2d_analytic=False, truncated=True)
 |
 |  `Navarro-Frenk-White
 |  <https://arxiv.org/abs/astro-ph/9508025>`_ profile.
 |
 |  .. math::
 |     \rho(r) = \frac{\rho_0}
 |     {\frac{r}{r_s}\left(1+\frac{r}{r_s}\right)^2}
 |
 |  where :math:`r_s` is related to the comoving spherical overdensity
 |  halo radius :math:`r_\Delta(M)` through the concentration
 |  parameter :math:`c(M)` as
 |
 |  .. math::
 |     r_\Delta(M) = c(M)\,r_s
 |
 |  and the normalization :math:`\rho_0` is
 |
 |  .. math::
 |     \rho_0 = \frac{M}{4\pi\,r_s^3\,[\log(1+c) - c/(1+c)]}
 |
 |  By default, this profile is truncated at :math:`r = r_\Delta(M)`.
 |
 |  Args:
 |      mass_def (:class:`~pyccl.halos.massdef.MassDef` or :obj:`str`):
 |          a mass definition object

# Setting up pars & profiles

In [5]:
hmd_200c = ccl.halos.MassDef200c # use a mass definition with Delta = 200
cM = ccl.halos.ConcentrationDuffy08(mass_def=hmd_200c)
cosmo = ccl.Cosmology(Omega_c = 0.25, Omega_b=0.05, h=0.07, A_s=2.1e-9, n_s=0.96, Neff=0, Omega_g=0)

trial_r = np.geomspace(1E-2,1E8,100000) # geomspace is like linspace, but evenly spaces on a logscale
trial_k = np.geomspace(1E-3,9E1,100000) # np.geomspace(1E-3,1E2,100000)
trial_M = np.array([3E12,6E13, 1E14])
trial_gamma = 1.1  # working with z = 0 (today) ; therefore a = 1 [default]
trial_f_bd = 0.6

## Loading profiles, with fourier_analytic = True

In [10]:
cdm_profile = BCMprofiles.CDMProfile(cosmo, hmd_200c, cM)
stel_profile = BCMprofiles.StellarProfile(cosmo, hmd_200c)
ej_profile = BCMprofiles.EjectedGasProfile(cosmo, hmd_200c)
bd_profile = BCMprofiles.BoundGasProfile(cosmo, hmd_200c, cM, trial_gamma)
bcm_profile = BCMprofiles.CombinedAllBCMProfile(cosmo, hmd_200c, cM, trial_gamma)

/Users/lambe/Documents/GitHub/Oxford-Intern-2024/Oxford-Internship-Practices/BCMprofiles_cleanUp.py:61: SyntaxWarning: invalid escape sequence '\ '
  """Creating a class for the stellar density profile
/Users/lambe/Documents/GitHub/Oxford-Intern-2024/Oxford-Internship-Practices/BCMprofiles_cleanUp.py:228: SyntaxWarning: invalid escape sequence '\ '
  """Creating a class for the bound gas density profile where:
/Users/lambe/Documents/GitHub/Oxford-Intern-2024/Oxford-Internship-Practices/BCMprofiles_cleanUp.py:435: SyntaxWarning: invalid escape sequence '\e'
  """Combined profile for the stellar & ejected & bound gas & cdm components (ie- The BCM Model), with the truncated Navarro-Frenk-White (NFW) profile used to calculate the density profiles of the cold dark matter (cdm) component.
/Users/lambe/Documents/GitHub/Oxford-Intern-2024/Oxford-Internship-Practices/BCMprofiles_cleanUp.py:504: SyntaxWarning: invalid escape sequence '\e'
  """Combined profile for the stellar & ejected & bound g

TypeError: HaloProfile.__init__() got an unexpected keyword argument 'fourier_analytic'

In [14]:
help(BCMprofiles.CDMProfile)

Help on class CDMProfile in module BCMprofiles_cleanUp:

class CDMProfile(pyccl.halos.profiles.profile_base.HaloProfile)
 |  CDMProfile(self, cosmo, mass_def, concentration, fourier_analytic=True, projected_analytic=False, cumul2d_analytic=False, truncated=True)
 |
 |  Density profile for the cold dark matter (cdm), using the Navarro-Frenk-White, multiplied by the cdm's mass fraction.
 |
 |  Method resolution order:
 |      CDMProfile
 |      pyccl.halos.profiles.profile_base.HaloProfile
 |      pyccl._core.schema.CCLAutoRepr
 |      pyccl._core.schema.CCLObject
 |      abc.ABC
 |      builtins.object
 |
 |  Methods defined here:
 |
 |  __init__(self, cosmo, mass_def, concentration, fourier_analytic=True, projected_analytic=False, cumul2d_analytic=False, truncated=True)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |
 |  update_parameters(self, cosmo=None, mass_def=None, concentration=None, fourier_analytic=None, projected_analytic=None, cumul2d_analytic=None,

In [24]:
stel_real = stel_profile._real(trial_r, trial_M)
ej_real = ej_profile._real(trial_r, trial_M)
bd_real = bd_profile._real(trial_r, trial_M)
bcm_real = bcm_profile._real(trial_r, trial_M)

In [ ]:
stel_fourier = stel_profile._fourier(trial_k, trial_M)
ej_fourier = ej_profile._fourier(trial_k, trial_M)
bd_fourier = bd_profile._fourier(trial_k, trial_M)
bcm_fourier = bcm_profile._fourier(trial_k, trial_M)

k = 1.56% through
k = 3.12% through
k = 4.69% through
k = 6.25% through
k = 7.81% through
k = 9.38% through
k = 10.9% through
k = 12.5% through
k = 14.1% through
k = 15.6% through
k = 17.2% through
k = 18.8% through
k = 20.3% through
k = 21.9% through
k = 23.4% through
k = 25% through
k = 26.6% through
k = 28.1% through
k = 29.7% through
k = 31.2% through
k = 32.8% through
k = 34.4% through
k = 35.9% through
k = 37.5% through
k = 39.1% through
k = 40.6% through
k = 42.2% through
k = 43.8% through
k = 45.3% through
k = 46.9% through
k = 48.4% through
k = 50% through
k = 51.6% through
k = 53.1% through
k = 54.7% through
k = 56.2% through
k = 57.8% through
k = 59.4% through
k = 60.9% through
k = 62.5% through
k = 64.1% through
k = 65.6% through
k = 67.2% through
k = 68.8% through
k = 70.3% through
k = 71.9% through
k = 73.4% through
k = 75% through
k = 76.6% through
k = 78.1% through
k = 79.7% through
k = 81.2% through
k = 82.8% through
k = 84.4% through
k = 85.9% through
k = 87.5% throug